In [ ]:
import pandas as pd
import sqlite3

# Load CSV

df_movies = pd.read_csv('/content/movies.csv',encoding ='latin1',on_bad_lines='skip', sep=';')  # path of your dataset

# Create SQLite DB file
conn = sqlite3.connect(':memory:')
df_movies.to_sql('movies', conn, index=False, if_exists='replace')

#conn.close()
print("CSV converted to DB successfully!")

CSV converted to DB successfully!


In [ ]:
print(df_movies.columns)

Index(['name', 'rating', 'genre', 'year', 'released', 'score', 'votes',
       'director', 'writer', 'star', 'country', 'budget', 'gross', 'company',
       'runtime'],
      dtype='object')


### Independent Subqueries

1. Show movie with max profit

In [ ]:
query="""
SELECT * FROM movies
WHERE gross-budget = (SELECT MAX(gross-budget) FROM movies)
"""
result = pd.read_sql_query(query, conn)
print(result)

      name rating  genre  year                           released  score  \
0  Titanic  PG-13  Drama  1997  December 19, 1997 (United States)    7.8   

       votes       director         writer               star        country  \
0  1100000.0  James Cameron  James Cameron  Leonardo DiCaprio  United States   

      budget         gross                company  runtime  
0  200000000  2.201647e+09  Twentieth Century Fox    194.0  


2. how many movies are above average

In [ ]:
query="""
SELECT COUNT(name) as movies_above_avg FROM movies
WHERE score > (SELECT AVG(score) FROM movies)
"""
result = pd.read_sql_query(query, conn)
print(result)

   movies_above_avg
0              2077


3. Find the highest rated movie of 2000

In [ ]:
query = """
SELECT name,score FROM movies
WHERE year = 2000 and score = (SELECT MAX(score) FROM movies
                               WHERE year = 2000)


"""
result = pd.read_sql_query(query, conn)
print(result)

        name  score
0  Gladiator    8.5


4. Find the highest rated movie among all movies whose number of votes are > the dataset avg votes

In [ ]:
query = """

SELECT name FROM movies
WHERE score = (SELECT MAX(score) FROM movies
               WHERE votes > (SELECT AVG(votes) FROM movies))

"""
result = pd.read_sql_query(query, conn)
print(result)

                       name
0  The Shawshank Redemption


5. Find all movies made by top 3 directors (in terms of total gross income)

In [ ]:
query = """

SELECT name FROM movies
WHERE director in (SELECT director FROM movies
                   GROUP BY director
                   ORDER BY SUM(gross) DESC LIMIT 3)

"""
result = pd.read_sql_query(query, conn)
print(result)

                                             name
0   Indiana Jones and the Raiders of the Lost Ark
1                      E.T. the Extra-Terrestrial
2                                  The Terminator
3            Indiana Jones and the Temple of Doom
4                             Romancing the Stone
5                              Back to the Future
6                                The Color Purple
7                                          Aliens
8                               Empire of the Sun
9                         Who Framed Roger Rabbit
10             Indiana Jones and the Last Crusade
11                                      The Abyss
12                     Back to the Future Part II
13                                         Always
14                    Back to the Future Part III
15                     Terminator 2: Judgment Day
16                                           Hook
17                              Death Becomes Her
18                                  Jurassic Park


 6. Find all movies of all those actors whose filmography's avg rating > 8.5  ( take 25000 votes as cutoff)

In [ ]:
query = """
SELECT name FROM movies
WHERE star IN (SELECT star FROM movies
               WHERE votes > 25000
               GROUP BY star
               HAVING AVG(score) > 8.5)
"""
result = pd.read_sql_query(query, conn)
print(result)

                                                name
0                                   Johnny Stecchino
1                        The Adventures of Huck Finn
2                            Son of the Pink Panther
3                                              North
4                                            The War
5                                  Life Is Beautiful
6  The Lord of the Rings: The Fellowship of the Ring
7                                      Spirited Away


7. Find the most profitable movie of each year

In [ ]:
query = """
SELECT name,year FROM movies
WHERE (year,(gross-budget)) in (SELECT year, MAX(gross-budget) FROM movies
                        GROUP BY year)

"""
result = pd.read_sql_query(query, conn)
print(result)

                                              name  year
0   Star Wars: Episode V - The Empire Strikes Back  1980
1    Indiana Jones and the Raiders of the Lost Ark  1981
2                       E.T. the Extra-Terrestrial  1982
3       Star Wars: Episode VI - Return of the Jedi  1983
4             Indiana Jones and the Temple of Doom  1984
5                               Back to the Future  1985
6                                          Top Gun  1986
7                                 Fatal Attraction  1987
8                                         Rain Man  1988
9               Indiana Jones and the Last Crusade  1989
10                                           Ghost  1990
11                      Terminator 2: Judgment Day  1991
12                                         Aladdin  1992
13                                   Jurassic Park  1993
14                                   The Lion King  1994
15                                       Toy Story  1995
16                             

In [ ]:
query = """
SELECT name,year,MAX(gross-budget) FROM movies
GROUP BY year

"""
result = pd.read_sql_query(query, conn)
print(result)

                                              name  year  MAX(gross-budget)
0   Star Wars: Episode V - The Empire Strikes Back  1980       5.203751e+08
1    Indiana Jones and the Raiders of the Lost Ark  1981       3.719260e+08
2                       E.T. the Extra-Terrestrial  1982       7.824106e+08
3       Star Wars: Episode VI - Return of the Jedi  1983       4.426062e+08
4             Indiana Jones and the Temple of Doom  1984       3.051073e+08
5                               Back to the Future  1985       3.629068e+08
6                                          Top Gun  1986       3.422882e+08
7                                 Fatal Attraction  1987       3.061457e+08
8                                         Rain Man  1988       3.298254e+08
9               Indiana Jones and the Last Crusade  1989       4.261718e+08
10                                           Ghost  1990       4.837036e+08
11                      Terminator 2: Judgment Day  1991       4.188812e+08
12          

8. Find the highest rated movie of each genre votes cutoff of 25000

In [ ]:
query = """
SELECT genre,name,score,votes FROM movies
WHERE (genre,score) IN (SELECT genre, MAX(score) FROM movies
                        WHERE votes > 25000
                        GROUP BY genre)
AND votes > 25000

"""
result = pd.read_sql_query(query,conn)
print(result)

        genre                                               name  score  \
0      Horror                                          The Thing    8.1   
1      Family                         E.T. the Extra-Terrestrial    7.8   
2     Romance                                            Starman    7.0   
3   Adventure                                 Back to the Future    8.5   
4   Biography                                   Schindler's List    8.9   
5       Drama                           The Shawshank Redemption    9.3   
6       Crime                                       Pulp Fiction    8.9   
7      Comedy                                  Life Is Beautiful    8.6   
8     Fantasy                                          Dark City    7.6   
9     Mystery                                            Memento    8.4   
10     Action  The Lord of the Rings: The Fellowship of the Ring    8.8   
11  Animation                                      Spirited Away    8.6   

        votes  
0    382

In [ ]:
query = """
SELECT genre, name,MAX(score),votes FROM movies
WHERE votes > 25000
GROUP BY genre
"""
result = pd.read_sql_query(query,conn)
print(result)

        genre                                               name  MAX(score)  \
0      Action  The Lord of the Rings: The Fellowship of the Ring         8.8   
1   Adventure                                 Back to the Future         8.5   
2   Animation                                      Spirited Away         8.6   
3   Biography                                   Schindler's List         8.9   
4      Comedy                                  Life Is Beautiful         8.6   
5       Crime                                       Pulp Fiction         8.9   
6       Drama                           The Shawshank Redemption         9.3   
7      Family                         E.T. the Extra-Terrestrial         7.8   
8     Fantasy                                          Dark City         7.6   
9      Horror                                          The Thing         8.1   
10    Mystery                                            Memento         8.4   
11    Romance                           

9. Find highest grossing movies of top 5 actor/director combo in terms of total gross income

In [ ]:
query = """

SELECT name FROM movies
WHERE (star,director,gross) IN (SELECT star,director,MAX(gross) FROM movies
GROUP BY star,director
ORDER BY SUM(gross) DESC LIMIT 5)



"""
result = pd.read_sql_query(query,conn)
print(result)


                                 name
0  Indiana Jones and the Last Crusade
1                     Lethal Weapon 3
2                        Forrest Gump
3                       The Lion King
4                             Titanic


### Correlated Subqueries

10. Find all those movies that have a rating higher than the average rating of movies in the same genre

In [ ]:
query = """
SELECT name FROM movies m1
WHERE score > (SELECT AVG(score) FROM movies m2  WHERE m2.genre = m1.genre)
"""
result = pd.read_sql_query(query,conn)
print(result)


                                                name
0                                        The Shining
1     Star Wars: Episode V - The Empire Strikes Back
2                                          Airplane!
3                                         Caddyshack
4                                    Friday the 13th
...                                              ...
2099                                Buffalo Soldiers
2100                           The Anniversary Party
2101                                        Blow Dry
2102                                    Human Nature
2103                                            Made

[2104 rows x 1 columns]


11. Get the percentage of votes for each movie compared to total percentage of votes

In [ ]:
query = """
SELECT name,(votes/(SELECT SUM(votes) FROM movies))*100 as percentage_votes FROM movies

"""
result = pd.read_sql_query(query,conn)
print(result)

                                                name  percentage_votes
0                                        The Shining          0.425272
1                                    The Blue Lagoon          0.029820
2     Star Wars: Episode V - The Empire Strikes Back          0.550514
3                                          Airplane!          0.101386
4                                         Caddyshack          0.049546
...                                              ...               ...
3995                                   Birthday Girl          0.011469
3996                                    Human Nature          0.008258
3997                                            Made          0.008258
3998                           One Night at McCool's          0.013304
3999                                  Dr. Dolittle 2          0.019268

[4000 rows x 2 columns]


12. Display all movie names, genre, score and avg(score) of genre

In [ ]:
query = """
SELECT name,genre,score,(SELECT AVG(score) FROM movies m2 WHERE m2.genre = m1.genre) FROM movies m1
"""
result = pd.read_sql_query(query,conn)
print(result)

                                                name      genre  score  \
0                                        The Shining      Drama    8.4   
1                                    The Blue Lagoon  Adventure    5.8   
2     Star Wars: Episode V - The Empire Strikes Back     Action    8.7   
3                                          Airplane!     Comedy    7.7   
4                                         Caddyshack     Comedy    7.3   
...                                              ...        ...    ...   
3995                                   Birthday Girl     Comedy    6.1   
3996                                    Human Nature     Comedy    6.4   
3997                                            Made     Comedy    6.4   
3998                           One Night at McCool's     Comedy    6.1   
3999                                  Dr. Dolittle 2     Comedy    4.7   

      (SELECT AVG(score) FROM movies m2 WHERE m2.genre = m1.genre)  
0                                         

13. Find genres having avg score > avg score of all the movies

In [ ]:
query = """
SELECT genre,AVG(score) FROM movies
GROUP BY genre
HAVING AVG(score) > (SELECT AVG(score) FROM movies)
"""
result = pd.read_sql_query(query,conn)
print(result)

       genre  AVG(score)
0  Animation    6.883186
1  Biography    7.052000
2      Crime    6.645033
3      Drama    6.673606
4     Family    6.550000
5    History    8.300000
6      Music    7.200000
7    Mystery    6.500000
8    Romance    6.400000
